In [1]:
import os
import glob
import re
import pandas as pd
import numpy as np
import PyPDF2

path = os.getcwd()
pdf_files = glob.glob(path + "/Protocolos/*.pdf")
print(len(pdf_files))


197


In [2]:
def convert_pdf_to_txt(path):
    pdf_file = open(path, 'rb')
    read_pdf = PyPDF2.PdfReader(pdf_file)
    number_of_pages = len(read_pdf.pages)
    page_content = ""
    for page_number in range(number_of_pages):
        page = read_pdf.pages[page_number]
        page_content += page.extract_text()
    pdf_file.close()
    return page_content
text = []
index=1
print("Proceso 1")
for i in pdf_files:
    print(f"{index}/{len(pdf_files)}", end='\r')
    text.append(convert_pdf_to_txt(i))
    index+=1

Proceso 1


In [24]:
def get_title(text):
    text = text.split('trabajo terminal')
    text = text[0]
    text = text.replace('\n', '')
    text = re.sub(' +', ' ', text)
    return text

def get_ttnum(text):
    text = text.split('trabajo terminal')
    text = text[1]
    # si alum aparece en ek texto
    if 'alumn' in text:
        text = text.split('alumn')
        text = text[0]
    if 'integrantes' in text:
        text = text.split('integrantes')
        text = text[0]
    text = re.sub(' +', '', text)
    # si aparece "no." en text remover
    if 'no.' in text:
        text = text.replace('no.', '')
    elif 'no:' in text:
        text = text.replace('no:', '')
    return text

def get_director(text):
    text = text.split('director')
    text = text[1]
    # si e-mail aparece en el texto 
    if 'e-mail' in text:
        text = text.split('e-mail')
        text = text[0]
    elif 'resumen' in text:
        text = text.split('resumen')
        text = text[0]
    # eliminar texto antes de :
    text = text.split(':')
    text = text[1]
    #eliminar :
    text = text.replace(':', '')
    text = text.replace('\n', '')
    text = re.sub(' +', ' ', text)
    return text

def get_abstract(text):
    text = text.split('resumen')
    text = text[1]
    text = text.split('palabras clave')
    text = text[0]
    text = text.replace('\n', '')
    text = re.sub(' +', ' ', text)
    return text

def get_keywords(text):
    text = text.split('palabras clave')
    text = text[1]
    text = text.split('.')
    text = text[0]
    text = re.sub(' +', ' ', text)
    return text

def get_areas(text):
    # si existe areas de interes en el texto 
    if 'áreas de interés' in text:
        text = text.split('áreas de interés')
        text = text[1]
        text = text.split('.')
        text = text[0]
        text = text.replace('\n', '')
        text = re.sub(' +', ' ', text)
        return text

def get_objetivo(text):
    text = text.split('objetivo')
    text = text[1]
    text = text.split('.')
    text = text[0]
    text = text.replace('\n', '')
    text = re.sub(' +', ' ', text)
    return text

def get_objetivos_especificos(text):
    # si existe "objetivos especificos" o "especificos" en el texto
    if 'objetivos específicos' in text or 'específicos' in text:
        if 'objetivos específicos' in text:
            text = text.split('objetivos específicos')
            text = text[1]
            text = text.split('.')
            text = text[0]
            text = text.replace('\n', '')
            text = re.sub(' +', ' ', text)
            return text
        else:
            text = text.split('específicos')
            text = text[1]
            text = text.split('.')
            text = text[0]
            text = text.replace('\n', '')
            text = re.sub(' +', ' ', text)
            return text

In [25]:
#pasar todos los text a minusculas 
text = [x.lower() for x in text]
# quitar todos los "\n" de text
text = [x.replace('\n', '') for x in text]
# quitar dobles espacios
text = [re.sub(' +', ' ', x) for x in text]

In [31]:
titulos = []
num_tt = []
abstracts = []
keywords = []
directores = []
areas_interes = []
objetivos = []
objetivos_especificos=[]
index=1
print("Proceso 2")
for j in text:
    print(f"{index}/{len(pdf_files)}", end='\r')
    titulos.append(get_title(j))
    # print(f"{titulos[index-1]}", end='\r')
    num_tt.append(get_ttnum(j))
    abstracts.append(get_abstract(j))
    keywords.append(get_keywords(j))
    directores.append(get_director(j))
    areas_interes.append(get_areas(j))
    objetivos.append(get_objetivo(j))
    objetivos_especificos.append(get_objetivos_especificos(j))
    index+=1
# obtener Objetivos específicos: hasta Justificación

Proceso 2


[' dr. suárez castañón miguel santiago, dr. carlos alberto duchanoy martínez,turno para la presentación del tt',
 ' vélez saldaña ulises , jiménez galán yasmín ivette *montes.romero@hotmail.com , sergalanm@gmail.com ',
 ' m. en c. zagal flores roberto eswart, dr. mata rivera miguel félix turno para la presentación del tt']

In [32]:
# guardar en un archivo de excel con las columnas titulo, N_TT, abstract, keywords, directores, objetivos
df = pd.DataFrame({'titulo': titulos , 'N_TT': num_tt,'Directores': directores})

# save the dataframe in a excel with the name "extraccion"
df.to_excel('extraccion.xlsx', index=False)

df.head()

,titulo,N_TT,Directores
0,prototipo de aplicación móvil para temas de f...,2019-b037,dra. ruíz ledesma elena fabiola *ocampos97@ou...
1,1 2 framework para la implementación de interf...,2019-a053,"pescador rojas miriam, coronilla contreras uk..."
2,monedero electronico para cafeterias escolares...,____-___,"dr. amadeo josé argüelles cruz, m. en ing. al..."
3,1 herramienta para la creación de un entorno p...,————-———,"carreto arellano chadwick , bustos farías edu..."
4,1 software de evaluación de datos de espectrom...,2016-b015,"dr. tonáhtiu arturo ramírez romero , miguel h..."


In [39]:
# procesar df con spacy 
import spacy
from spacy.lang.es.examples import sentences 
nlp = spacy.load("es_core_news_sm")
# how to install spacy 
# https://spacy.io/usage/models#languages


# para cada titulo de los titulos procesar con nlp
for i in df['titulo']:
    doc = nlp(i)
    # para cada token en el doc 
    for token in doc:
        # si el token es un verbo 
        if token.pos_ == 'VERB':
            print(token.text)


OSError: [E050] Can't find model 'es_core_news_sm'. It doesn't seem to be a Python package or a valid path to a data directory.